In [5]:
import pandas as pd
from IPython.display import Markdown

df = pd.read_csv("../results/trace_resolved.csv")
df.head()

,timestamp_ns,thread_id,event,callee_addr,callee_func,callee_fileline,caller_addr,caller_func,caller_fileline
0,1751729557632068189,51361,ENTER,0x402909,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...,0x40901d,__libc_csu_init,??:?
1,1751729557632126794,51361,ENTER,0x40286a,"__static_initialization_and_destruction_0(int,...",/home/nhorro/workspace/learn-code-analysis/01_...,0x402933,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...
2,1751729557632263852,51361,EXIT,0x40286a,"__static_initialization_and_destruction_0(int,...",/home/nhorro/workspace/learn-code-analysis/01_...,0x402933,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...
3,1751729557632270458,51361,EXIT,0x402909,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...,0x40901d,__libc_csu_init,??:?
4,1751729557632275265,51361,ENTER,0x40260b,main,/home/nhorro/workspace/learn-code-analysis/01_...,0x7ff21a89a083,??,??:0


In [7]:
import re

def is_relevant(func):
    if func.startswith("main"): return True
    if func.startswith("Manager::"): return True
    if func.startswith("Worker::"): return True
    if func.startswith("Reporter::"): return True
    return False

df_logic = df[df['callee_func'].apply(is_relevant)]

In [8]:
df_logic

,timestamp_ns,thread_id,event,callee_addr,callee_func,callee_fileline,caller_addr,caller_func,caller_fileline
4,1751729557632275265,51361,ENTER,0x40260b,main,/home/nhorro/workspace/learn-code-analysis/01_...,0x7ff21a89a083,??,??:0
5,1751729557632305560,51361,ENTER,0x403c52,Manager::Manager(),/home/nhorro/workspace/learn-code-analysis/01_...,0x402658,main,/home/nhorro/workspace/learn-code-analysis/01_...
6,1751729557632310937,51361,ENTER,0x4031ea,Worker::Worker(),/home/nhorro/workspace/learn-code-analysis/01_...,0x403c91,Manager::Manager(),/home/nhorro/workspace/learn-code-analysis/01_...
87,1751729557632669155,51361,EXIT,0x4031ea,Worker::Worker(),/home/nhorro/workspace/learn-code-analysis/01_...,0x403c91,Manager::Manager(),/home/nhorro/workspace/learn-code-analysis/01_...
108,1751729557632762329,51361,ENTER,0x4035aa,Worker::onJobDone(std::function<void (CalcJob ...,/home/nhorro/workspace/learn-code-analysis/01_...,0x403cb7,Manager::Manager(),/home/nhorro/workspace/learn-code-analysis/01_...
...,...,...,...,...,...,...,...,...,...
939,1751729558636225652,51361,ENTER,0x407324,Manager::Manager()::{lambda(CalcJob const&)#1}...,/usr/include/c++/9/bits/std_function.h:92,0x40698d,std::_Function_base::_Base_manager<Manager::Ma...,/usr/include/c++/9/bits/std_function.h:184
942,1751729558636239942,51361,EXIT,0x407324,Manager::Manager()::{lambda(CalcJob const&)#1}...,/usr/include/c++/9/bits/std_function.h:92,0x40698d,std::_Function_base::_Base_manager<Manager::Ma...,/usr/include/c++/9/bits/std_function.h:184
1007,1751729558636585764,51361,EXIT,0x403b52,Worker::~Worker(),/home/nhorro/workspace/learn-code-analysis/01_...,0x403ed3,Manager::~Manager(),/home/nhorro/workspace/learn-code-analysis/01_...
1008,1751729558636590401,51361,EXIT,0x403ea4,Manager::~Manager(),/home/nhorro/workspace/learn-code-analysis/01_...,0x4027f8,main,/home/nhorro/workspace/learn-code-analysis/01_...


In [10]:
from IPython.display import Markdown
import pandas as pd
import re

# --- Función robusta para extraer clase/namespace principal ---
def extract_participant(func):
    if not isinstance(func, str) or func == "??":
        return "Global"
    match = re.search(r'([a-zA-Z_][\w]*)::', func)
    if match:
        part = match.group(1)
    else:
        part = func.split("::")[0] if "::" in func else func
    if part.startswith("std") or " " in part:
        return None
    return part

# --- 1️⃣ Identificar participantes únicos ---
participants = set()

for func in pd.concat([df_logic['caller_func'], df_logic['callee_func']]):
    part = extract_participant(func)
    if part:
        participants.add(part)

participants = sorted(participants)
print("Participants:", participants)

# --- 2️⃣ Generar código PlantUML ---
uml = ["@startuml"]

# Agregar participantes
for part in participants:
    uml.append(f"participant {part}")

# --- 3️⃣ Insertar llamadas con notas por hilo ---
last_tid = None

for _, row in df_logic.sort_values('timestamp_ns').iterrows():
    tid = row['thread_id']
    event = row['event']
    caller = extract_participant(row['caller_func']) or "Global"
    callee = extract_participant(row['callee_func']) or "Global"
    func = row['callee_func']

    if tid != last_tid:
        uml.append(f"note over {caller}: Thread {tid}")
        last_tid = tid

    if event == "ENTER":
        uml.append(f"{caller} -> {callee}: {func}")
    elif event == "EXIT":
        uml.append(f"{callee} --> {caller}: return")

uml.append("@enduml")

# --- 4️⃣ Mostrar en notebook para copy-paste ---
plantuml_code = "\n".join(uml)
display(Markdown(f"```plantuml\n{plantuml_code}\n```"))

Participants: ['Global', 'Manager', 'Reporter', 'Worker', 'main']


```plantuml
@startuml
participant Global
participant Manager
participant Reporter
participant Worker
participant main
note over Global: Thread 51361
Global -> main: main
main -> Manager: Manager::Manager()
Manager -> Worker: Worker::Worker()
Worker --> Manager: return
Manager -> Worker: Worker::onJobDone(std::function<void (CalcJob const&)>)
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>() const
Manager --> Global: return
Worker --> Manager: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1}& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>()
Manager --> Global: return
Manager --> main: return
main -> Manager: Manager::start()
Manager -> Worker: Worker::start()
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
note over Global: Thread 51362
Global -> Worker: Worker::start()::{lambda()#1}& std::__get_helper<0ul, Worker::start()::{lambda()#1}>(std::_Tuple_impl<0ul, Worker::start()::{lambda()#1}>&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}&&>(std::remove_reference<Worker::start()::{lambda()#1}&&>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)
note over Manager: Thread 51361
Worker --> Manager: return
note over Global: Thread 51362
Worker --> Global: return
note over main: Thread 51361
Manager --> main: return
note over Global: Thread 51362
Global -> Worker: Worker::start()::{lambda()#1}::operator()() const
note over main: Thread 51361
main -> Manager: Manager::submitJob(CalcJob const&)
note over Worker: Thread 51362
Worker -> Worker: Worker::run()
note over Manager: Thread 51361
Manager -> Worker: Worker::enqueueJob(CalcJob const&)
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::submitJob(CalcJob const&)
Manager -> Worker: Worker::enqueueJob(CalcJob const&)
note over Global: Thread 51362
Global -> Worker: Worker::run()::{lambda()#1}::operator()() const
Worker --> Global: return
Worker -> Worker: Worker::process(CalcJob&)
Worker --> Worker: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>() const
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const* std::__addressof<Manager::Manager()::{lambda(CalcJob const&)#1} const>(Manager::Manager()::{lambda(CalcJob const&)#1} const&)
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1}::operator()(CalcJob const&) const
Manager -> Reporter: Reporter::report(CalcJob const&)
note over Manager: Thread 51361
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::submitJob(CalcJob const&)
note over Reporter: Thread 51362
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)
note over Manager: Thread 51361
Manager -> Worker: Worker::enqueueJob(CalcJob const&)
note over Reporter: Thread 51362
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Global: return
note over Manager: Thread 51361
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::submitJob(CalcJob const&)
Manager -> Worker: Worker::enqueueJob(CalcJob const&)
Worker --> Manager: return
Manager --> main: return
note over Global: Thread 51362
Global -> Worker: Worker::run()::{lambda()#1}::operator()() const
Worker --> Global: return
Worker -> Worker: Worker::process(CalcJob&)
Worker --> Worker: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>() const
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const* std::__addressof<Manager::Manager()::{lambda(CalcJob const&)#1} const>(Manager::Manager()::{lambda(CalcJob const&)#1} const&)
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1}::operator()(CalcJob const&) const
Manager -> Reporter: Reporter::report(CalcJob const&)
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Global: return
Global -> Worker: Worker::run()::{lambda()#1}::operator()() const
Worker --> Global: return
Worker -> Worker: Worker::process(CalcJob&)
Worker --> Worker: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>() const
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const* std::__addressof<Manager::Manager()::{lambda(CalcJob const&)#1} const>(Manager::Manager()::{lambda(CalcJob const&)#1} const&)
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1}::operator()(CalcJob const&) const
Manager -> Reporter: Reporter::report(CalcJob const&)
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Global: return
Global -> Worker: Worker::run()::{lambda()#1}::operator()() const
Worker --> Global: return
Worker -> Worker: Worker::process(CalcJob&)
Worker --> Worker: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>() const
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1} const* std::__addressof<Manager::Manager()::{lambda(CalcJob const&)#1} const>(Manager::Manager()::{lambda(CalcJob const&)#1} const&)
Manager --> Global: return
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1}::operator()(CalcJob const&) const
Manager -> Reporter: Reporter::report(CalcJob const&)
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Global: return
Global -> Worker: Worker::run()::{lambda()#1}::operator()() const
Worker --> Global: return
note over main: Thread 51361
main -> Manager: Manager::stop()
Manager -> Worker: Worker::stopWorker()
note over Global: Thread 51362
Global -> Worker: Worker::run()::{lambda()#1}::operator()() const
Worker --> Global: return
Worker --> Worker: return
Worker --> Global: return
note over Manager: Thread 51361
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::~Manager()
Manager -> Worker: Worker::~Worker()
Global -> Manager: Manager::Manager()::{lambda(CalcJob const&)#1}& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>()
Manager --> Global: return
Worker --> Manager: return
Manager --> main: return
main --> Global: return
@enduml
```